In [4]:
import affine
import xarray as xr
import numpy as np
from collections.abc import Iterable
from collections.abc import Hashable
from collections.abc import Mapping
from typing import Any


class Affine2DCoordinateTransform(xr.CoordinateTransform):
    """Affine 2D coordinate transform."""

    affine: affine.Affine
    xy_dims = tuple[str]
    
    def __init__(
        self,
        affine: affine.Affine,
        coord_names: Iterable[Hashable],
        dim_size: Mapping[str, int],
        dtype: Any = np.dtype(np.float64),
    ):
        # two dimensions
        assert len(coord_names) == 2
        assert len(dim_size) == 2

        super().__init__(coord_names, dim_size, dtype=dtype)
        self.affine = affine

        # array dimensions in reverse order (y = rows, x = cols)
        self.xy_dims = tuple(self.dims)
        self.dims = (self.dims[1], self.dims[0])

    def forward(self, dim_positions):
        positions = [dim_positions[dim] for dim in self.xy_dims]
        x_labels, y_labels = self.affine * tuple(positions)

        results = {}
        for name, labels in zip(self.coord_names, [x_labels, y_labels]):
            results[name] = labels

        return results

    def reverse(self, coord_labels):
        labels = [coord_labels[name] for name in self.coord_names]
        x_positions, y_positions = ~self.affine * tuple(labels)

        results = {}
        for dim, positions in zip(self.xy_dims, [x_positions, y_positions]):
            results[dim] = positions

        return results
    
    def equals(self, other):
        return self.affine == other.affine and self.dim_size == other.dim_size


from xarray.indexes import CoordinateTransformIndex

transform = Affine2DCoordinateTransform(
    affine.Affine.scale(1.0, 1.0),
    coord_names=("xc", "yc"),
    dim_size={"x": 5, "y": 5},
)

index = CoordinateTransformIndex(transform)
ds = xr.Dataset(coords=index.create_coordinates())


ds.yc.values

array([[0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.],
       [2., 2., 2., 2., 2.],
       [3., 3., 3., 3., 3.],
       [4., 4., 4., 4., 4.]])

In [5]:
# create an Affine from shape and (optionally) bbox
def from_dim(dim, bbox = None): 
    if bbox is None: 
        bbox = (0.0, 0.0, dim[0], dim[1])
    gdal = (bbox[0], (bbox[2] - bbox[0]) / dim[0], 0.0, 
             bbox[3], 0.0, (bbox[1] - bbox[3]) / dim[1] )
    return(affine.Affine.from_gdal(*gdal))

bbox = (-180, -90, 180, 90)
shape = (4, 2)  ## a 4x2 chunk world
from_dim(shape, bbox)

Affine(90.0, 0.0, -180.0,
       0.0, -90.0, 90.0)

In [6]:
from xarray.indexes import CoordinateTransformIndex

transform = Affine2DCoordinateTransform(
    from_dim(shape, bbox),
    coord_names=("xc", "yc"),
    dim_size={"x": shape[0], "y": shape[1]},
)

index = CoordinateTransformIndex(transform)
ds = xr.Dataset(coords=index.create_coordinates())


print(ds.xc.values)
print(ds.yc.values)

## compare to the cell centres
af = from_dim(shape, bbox)

print([x + af.a/2 for x in ds.xc.values])
print([y + af.e/2 for y in ds.yc.values])



[[-180.  -90.    0.   90.]
 [-180.  -90.    0.   90.]]
[[90. 90. 90. 90.]
 [ 0.  0.  0.  0.]]
[array([-135.,  -45.,   45.,  135.]), array([-135.,  -45.,   45.,  135.])]
[array([45., 45., 45., 45.]), array([-45., -45., -45., -45.])]


af

In [7]:
## sea ice example
bbox = (-3950000, -3950000,3950000, 4350000)
shape = (316, 332)

from xarray.indexes import CoordinateTransformIndex

transform = Affine2DCoordinateTransform(
    from_dim(shape, bbox),
    coord_names=("xc", "yc"),
    dim_size={"x": shape[1], "y": shape[0]},
)

index = CoordinateTransformIndex(transform)
ds = xr.Dataset(coords=index.create_coordinates())

ds.isel(x = slice(0, 2), y = slice(0, 4))

<xarray.Dataset> Size: 128B
Dimensions:  (y: 4, x: 2)
Coordinates:
    xc       (y, x) float64 64B -3.95e+06 -3.925e+06 ... -3.95e+06 -3.925e+06
    yc       (y, x) float64 64B 4.35e+06 4.35e+06 ... 4.275e+06 4.275e+06
Dimensions without coordinates: y, x
Data variables:
    *empty*

In [9]:
dsn = "/vsicurl/https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif"
ice = xr.load_dataset(dsn, engine = "rasterio")

ice.isel(x = slice(0, 159), y = slice(0, 167))

<xarray.Dataset> Size: 109kB
Dimensions:      (band: 1, x: 159, y: 167)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 1kB -3.938e+06 -3.912e+06 ... -1.25e+04 1.25e+04
  * y            (y) float64 1kB 4.338e+06 4.312e+06 ... 2.125e+05 1.875e+05
    spatial_ref  int64 8B 0
Data variables:
    band_data    (band, y, x) float32 106kB 0.0 0.0 0.0 ... 2.54e+03 2.54e+03

In [10]:
ds = xr.DataArray(ice.band_data.squeeze().values, coords=index.create_coordinates())
ds.isel(x = slice(0, 159), y = slice(0, 167))

<xarray.DataArray (y: 167, x: 159)> Size: 106kB
array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ..., 2540., 2540., 2540.],
       [   0.,    0.,    0., ..., 2540., 2540., 2540.],
       [   0.,    0.,    0., ..., 2540., 2540., 2540.]], dtype=float32)
Coordinates:
    xc       (y, x) float64 212kB -3.95e+06 -3.925e+06 -3.9e+06 ... -2.5e+04 0.0
    yc       (y, x) float64 212kB 4.35e+06 4.35e+06 4.35e+06 ... 2e+05 2e+05
Dimensions without coordinates: y, x